In [ ]:
def reverse_geocode(lat, lon):
    response = requests.get(NOMINATIM_URL, params={
        "lat": lat,
        "lon": lon,
        "format": "json",
        "zoom": 17,
        "addressdetails": 1
    }, headers=HEADERS)
    return response.json()

def get_elevation(lat, lon):
    response = requests.get(f"{ELEVATION_URL}?locations={lat},{lon}")
    result = response.json()
    return result['results'][0]['elevation']

def overpass_query(lat, lon, feature):
    if feature == "mountain":
        query = f"""
        [out:json][timeout:25];
        (node(around:3000,{lat},{lon})["natural"~"peak|hill"];
         way(around:3000,{lat},{lon})["natural"~"peak|hill"];);
        out center;
        """
    elif feature == "water":
        query = f"""
        [out:json][timeout:25];
        (node(around:3000,{lat},{lon})["natural"~"coastline|bay|beach|water|sea"];
         way(around:3000,{lat},{lon})["natural"~"coastline|bay|beach|water|sea"];
         relation(around:3000,{lat},{lon})["natural"~"coastline|bay|beach|water|sea"];);
        out center;
        """
    else:
        return []
    response = requests.post(OVERPASS_URL, data=query)
    return response.json().get("elements", [])

def analyze_location(lat, lon):
    info = reverse_geocode(lat, lon)
    display_name = info.get('display_name', '未知地名')
    elevation = get_elevation(lat, lon)
    has_mountain = len(overpass_query(lat, lon, "mountain")) > 0
    has_water = len(overpass_query(lat, lon, "water")) > 0
    return {
        "display_name": display_name,
        "elevation": elevation,
        "has_mountain": has_mountain,
        "has_water": has_water
    }
